In [ ]:
!pip -q install openai==1.* sentence-transformers faiss-cpu pandas tqdm pymupdf pillow requests

In [ ]:
import os, re, io, json, base64, importlib.util
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import requests, fitz  # PyMuPDF
from PIL import Image, ImageEnhance, ImageOps

In [ ]:
##import os
#os.environ["OPENAI_API_KEY"] = ""
#My Google collab

import os
from openai import OpenAI

api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Missing OPENAI_API_KEY environment variable. Please set it before running.")

client = OpenAI(api_key=api_key)

In [ ]:
Image.MAX_IMAGE_PIXELS = None

CSV_URL = "https://raw.githubusercontent.com/anniedoris/design_qa/main/dataset/rule_compliance/rule_dimension_qa/detailed_context/rule_dimension_qa_detailed_context.csv"
df = pd.read_csv(CSV_URL)
print("CSV rows:", len(df))

download_dir = Path("/content/detailed_context_images")
download_dir.mkdir(parents=True, exist_ok=True)
BASE_IMG_URL = "https://raw.githubusercontent.com/anniedoris/design_qa/main/dataset/rule_compliance/rule_dimension_qa/detailed_context/"

for fname in df['image'].unique():
    p = download_dir / fname
    if not p.exists():
        r = requests.get(BASE_IMG_URL + fname, timeout=30)
        if r.status_code == 200:
            p.write_bytes(r.content)
print("Images ready:", len(list(download_dir.iterdir())))

from google.colab import files
print("Upload FSAE_Rules_2024_V1.pdf …")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

In [ ]:
start_page = 4
doc = fitz.open(pdf_path)

def extract_text_from_page(p): return p.get_text("text")

HEADER_RES = [
    re.compile(r'^\s*Formula SAE.*Page\s+\d+\s+of\s+\d+\s*$', re.I),
    re.compile(r'^\s*Version\s+\d+(\.\d+)?\s+\d{1,2}\s+\w+\s+\d{4}\s*$', re.I),
    re.compile(r'^\s*\d+\s*$', re.I),
]
TOC_LINE_RE = re.compile(r'.+\.\s?\.\s?\.\s+\d+$')
SECTION_BANNER_RE = re.compile(r'^[A-Z]{1,4}\s*-\s+.+$')

def clean_lines(lines):
    out = []
    for ln in lines:
        s = ln.rstrip().replace('\xa0',' ')
        if any(rx.match(s) for rx in HEADER_RES): continue
        if TOC_LINE_RE.search(s): continue
        if SECTION_BANNER_RE.match(s): continue
        out.append(s)
    return out

pages = []
for i in range(start_page, len(doc)):
    lines = extract_text_from_page(doc[i]).splitlines()
    pages.append("\n".join(clean_lines(lines)))

raw_text = "\n".join(pages)

def clean_text(text):
    text = text.replace('\xa0',' ')
    text = re.sub(r'-\n','', text)
    text = re.sub(r'\n{3,}','\n\n', text)
    text = re.sub(r'[ \t]+',' ', text)
    return text.strip()

full_text = clean_text(raw_text)

# --- Parse rules ---
RULE_HEAD_RE = re.compile(r'(?m)^(?P<rid>[A-Z]{1,4}\.\d+(?:\.\d+)*)(?:[ \t]+(?P<title>.+))?$')

def parse_rules_from_text(text):
    lines = text.splitlines()
    rules, cur_id, buf = [], None, []

    def flush():
        nonlocal cur_id, buf, rules
        if cur_id and buf:
            content = "\n".join(buf).strip()
            if content and not content.startswith(cur_id):
                content = f"{cur_id} " + content
            rules.append((cur_id, content))
        cur_id, buf = None, []

    for ln in lines:
        s = ln.strip()
        m = RULE_HEAD_RE.match(s)
        if m:
            flush()
            cur_id = m.group("rid").strip()
            buf = [s]
        elif cur_id:
            buf.append(ln)
    flush()
    return rules

rule_pairs = parse_rules_from_text(full_text)
rule_chunks = {rid: txt for rid, txt in rule_pairs}
print("Total rules parsed:", len(rule_chunks))

with open("/content/rule_chunks.json","w",encoding="utf-8") as f:
    json.dump(rule_chunks, f, indent=2, ensure_ascii=False)
pd.DataFrame([{"rule_id":k,"chars":len(v),"text":v} for k,v in rule_chunks.items()])\
  .sort_values("rule_id").to_csv("/content/rule_chunks_preview.csv", index=False)
print("Wrote rule_chunks.json + rule_chunks_preview.csv")

In [ ]:
import faiss
from sentence_transformers import SentenceTransformer

rule_ids   = list(rule_chunks.keys())
rule_texts = [f"{rid}: {rule_chunks[rid]}" for rid in rule_ids]

embedder = SentenceTransformer('all-MiniLM-L6-v2')
rule_emb = embedder.encode(rule_texts, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
index = faiss.IndexFlatIP(rule_emb.shape[1])
index.add(rule_emb)

def retrieve_rule_candidates(query, top_k=5):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q, top_k)
    return [(rule_ids[i], rule_texts[i]) for i in I[0]]

def retrieve_exact_rule_or_fallback(question, top_k=5):
    ids_in_q = re.findall(r'[A-Z]{1,4}\.\d+(?:\.\d+)*(?:[a-z])?', question)
    primary = None
    for cand in ids_in_q:
        if cand in rule_chunks and len(rule_chunks[cand]) > 40:
            primary = (cand, f"{cand}: {rule_chunks[cand]}")
            break
    cands = retrieve_rule_candidates(question, top_k=top_k)
    if primary:
        cands = [x for x in cands if x[0] != primary[0]]
        cands.insert(0, primary)
    return cands[:top_k]

def load_image(path, max_side=2400):
    im = Image.open(path)
    im = ImageOps.exif_transpose(im).convert("RGB")
    w, h = im.size
    if max(w, h) > max_side:
        scale = max_side / max(w, h)
        im = im.resize((int(w*scale), int(h*scale)), Image.LANCZOS)
    return im

def enhance_for_reading(img: Image.Image, upscale=1.5):
    w, h = img.size
    img2 = img.resize((int(w*upscale), int(h*upscale)), Image.LANCZOS)
    img2 = ImageEnhance.Color(img2).enhance(0.0)
    img2 = ImageEnhance.Contrast(img2).enhance(1.6)
    img2 = ImageEnhance.Sharpness(img2).enhance(1.8)
    return img2

def make_top_bar_crop(img: Image.Image, frac=0.30):
    w, h = img.size
    return img.crop((0, 0, w, int(h*frac)))

def to_b64(img, fmt="JPEG", quality=95):
    buf = io.BytesIO()
    img.save(buf, format=fmt, quality=quality)
    return base64.b64encode(buf.getvalue()).decode("utf-8")

def robust_json_extract(text):
    m = re.search(r'\{.*\}', str(text), re.S)
    if m:
        try: return json.loads(m.group(0))
        except: pass
    mv = re.search(r'(\d+(?:\.\d+)?)\s*mm', str(text))
    return {"answer":"PARSE_ERROR","measured_value": (mv.group(0) if mv else None),
            "rule_id":None,"explanation":str(text)[:500]}

def parse_mm(s):
    if s is None: return None
    mm = re.search(r'(\d+(?:\.\d+)?)\s*mm', str(s).replace(",", ""))
    return float(mm.group(1)) if mm else None

def parse_threshold(rule_text):
    t = rule_text.lower().replace(",", "")
    nums = list(re.finditer(r'(\d+(?:\.\d+)?)\s*mm', t))
    if not nums: return None, None
    def sense(m):
        pos = m.start(); w = t[max(0,pos-100):pos+100]
        if any(k in w for k in ["minimum","at least","no less","≥",">="]): return (">=", float(m.group(1)))
        if any(k in w for k in ["maximum","at most","no more","≤","<="]):   return ("<=", float(m.group(1)))
        if any(k in w for k in ["exactly","equal to","equals","must be"]):  return ("==", float(m.group(1)))
        return None
    for m in nums:
        s = sense(m)
        if s: return s
    return (">=", float(nums[0].group(1)))

def radius_to_diameter_if_needed(text_near_value, measured_mm):
    return measured_mm * 2.0 if ("radius" in (text_near_value or "").lower() and "diameter" not in (text_near_value or "").lower() and measured_mm is not None) else measured_mm

def decide_answer(rule_text, measured_value_str, text_for_hints=None, tol=0.5):
    op, thr = parse_threshold(rule_text)
    meas = parse_mm(measured_value_str)
    meas = radius_to_diameter_if_needed(text_for_hints, meas)
    if op is None or thr is None or meas is None:
        return "CANNOT BE DETERMINED", {"op":op,"thr":thr,"meas":meas,"why":"missing data"}
    if op == ">=": ans = "Yes" if meas >= thr - tol else "No"
    elif op == "<=": ans = "Yes" if meas <= thr + tol else "No"
    elif op == "==": ans = "Yes" if abs(meas - thr) <= tol else "No"
    else: ans = "CANNOT BE DETERMINED"
    return ans, {"op":op,"thr":thr,"meas":meas}

In [ ]:
SYSTEM_PROMPT = "You are an FSAE rule compliance checker. Use ONLY the engineering drawing(s) and the rule text."

def ask_model(question, image_b64_list, rules_joined, model="gpt-4o"):
    user_prompt = f"""
Rule(s):
{rules_joined}

Task:
1) Identify EXACTLY the dimension relevant to the rule (name it).
2) Quote the dimension text exactly as shown on the drawing.
3) If the drawing shows radius but the rule uses diameter, compute diameter = 2 × radius.
4) If the question mentions a scale bar, use it and state the pixel→mm conversion.
5) Compare the measured value with the rule requirement and decide compliance.

Output JSON ONLY:
{{
  "answer": "Yes" | "No" | "CANNOT BE DETERMINED",
  "measured_value": "<number> mm",
  "rule_id": "<best matching rule id>",
  "explanation": "<1–3 sentence; include the quoted dimension label>"
}}
"""
    content = [{"type":"text","text": user_prompt}]
    for b64 in image_b64_list:
        content.append({"type":"image_url","image_url":{"url":f"data:image/jpeg;base64,{b64}","detail":"high"}})

    resp = client.chat.completions.create(
        model=model, temperature=0, max_tokens=450,
        messages=[{"role":"system","content": SYSTEM_PROMPT},
                  {"role":"user","content": content}]
    )
    raw = resp.choices[0].message.content or ""
    return robust_json_extract(raw), raw

results = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    q = row['question']
    img_file = row['image']
    gt = str(row['ground_truth']).strip().lower()

    p = download_dir / img_file
    if not p.exists():
        print("Missing:", p); continue

    img = load_image(p)
    img_enh = enhance_for_reading(img, upscale=1.5)
    images_b64 = [to_b64(img), to_b64(img_enh)]
    if "scale bar" in q.lower():
        images_b64.append(to_b64(make_top_bar_crop(img_enh, frac=0.32)))

    retrieved = retrieve_exact_rule_or_fallback(q, top_k=5)
    rules_joined = "\n".join([rt for rid, rt in retrieved])[:1400]

    mdict, raw_text = ask_model(q, images_b64, rules_joined, model="gpt-4o")
    final_answer, audit = decide_answer(rules_joined, mdict.get("measured_value"), mdict.get("explanation",""))

    model_prediction_text = f"Explanation: {mdict.get('explanation','').strip()} Answer: {final_answer}"

    results.append({
        "idx": idx, "question": q, "image": img_file,
        "ground_truth": gt, "retrieved_rule_top0": retrieved[0][1] if retrieved else "",
        "raw_model_text": raw_text[:1000], "measured_value": mdict.get("measured_value"),
        "answer_model": mdict.get("answer"), "answer_final": final_answer,
        "audit": audit, "model_prediction": model_prediction_text
    })


In [ ]:

res_df = pd.DataFrame(results)
res_df.to_csv("/content/dimension_full_predictions.csv", index=False)
print("Wrote: /content/dimension_full_predictions.csv")


In [ ]:
import sys, subprocess, re
import pandas as pd
from pathlib import Path

try:
    PRED = res_df.copy()
except NameError:
    PRED = pd.read_csv("/content/dimension_full_predictions.csv").copy()

if not Path("/content/design_qa").exists():
    subprocess.run(
        ["git", "clone", "-q",
         "https://github.com/anniedoris/design_qa.git",
         "/content/design_qa"],
        check=True
    )
sys.path.append("/content/design_qa")


subprocess.run([sys.executable, "-m", "pip", "install", "-q", "rouge", "nltk"], check=True)
import nltk; nltk.download('punkt', quiet=True)
GT = pd.read_csv(CSV_URL).copy()
if "idx" not in PRED.columns:
    PRED = PRED.reset_index().rename(columns={"index":"idx"})
if "idx" not in GT.columns:
    GT = GT.reset_index().rename(columns={"index":"idx"})

J = GT.merge(PRED, on="idx", how="inner", suffixes=("_gt", "_pred"))
missing = len(GT) - len(J)
if missing > 0:
    print(f"[WARN] {missing} GT rows had no matching prediction by idx.")
YES, NO, CBD = "yes", "no", "cannot be determined"

def _norm_truth(s: str) -> str:
    t = (s or "").strip().lower()
    if t.startswith("y"): return YES
    if t.startswith("n"): return NO
    return CBD

def _norm_pred(s: str) -> str:
    t = (s or "").strip().lower()
    m = re.search(r"\banswer\s*:\s*(yes|no|cannot be determined)\b", t, re.I)
    if m: t = m.group(1).lower()
    if "cannot" in t: return CBD
    if t.startswith("y"): return YES
    if t.startswith("n"): return NO
    return CBD

def pick_col(df, *candidates):
    for c in candidates:
        if c in df.columns:
            return c
    raise KeyError(f"None of the expected columns found. Looked for: {candidates}\nHave: {list(df.columns)}")

gt_col        = pick_col(J, "ground_truth_gt", "ground_truth", "GroundTruth", "answer", "gt", "expected")
pred_col      = "answer_final" if "answer_final" in J.columns else pick_col(J, "model_prediction_pred", "model_prediction")
raw_col       = "raw_model_text" if "raw_model_text" in J.columns else None
dimtype_col   = pick_col(J, "dimension_type_gt", "dimension_type")
expl_gt_col   = pick_col(J, "explanation_gt", "explanation")
J["ground_truth_norm"]   = J[gt_col].apply(_norm_truth)
J["model_prediction_ok"] = J[pred_col].apply(_norm_pred)

EVAL_DIR   = Path("/content/results"); EVAL_DIR.mkdir(parents=True, exist_ok=True)
DIM_EVAL   = str(EVAL_DIR / "dimension_eval_official.csv")
DIM_TWOCOL = str(EVAL_DIR / "dimension_for_full_eval.csv")

# Official-style CSV
df_eval = pd.DataFrame({
    "ground_truth":    J["ground_truth_norm"],
    "model_prediction": J.apply(
        lambda r: f"Explanation: {str(r.get(raw_col,''))[:200]} "
                  f"Answer: {r['model_prediction_ok']}",
        axis=1
    ),
    "dimension_type":  J[dimtype_col],
    "explanation":     J[expl_gt_col],
})
df_eval.to_csv(DIM_EVAL, index=False)
print(f"[OK] Prepared official eval CSV → {DIM_EVAL}")
pd.DataFrame({
    "ground_truth":    J["ground_truth_norm"],
    "model_prediction": J["model_prediction_ok"]
}).to_csv(DIM_TWOCOL, index=False)
print(f"[OK] Two-column GT vs Prediction → {DIM_TWOCOL}")

from eval.metrics.metrics import eval_dimensions_qa
acc_macro, direct_avg, scale_avg, *_ = eval_dimensions_qa(DIM_EVAL)
print("\nDimension")
print(f"(ACC ↑)\t\t{acc_macro:.3f}")

with open("/content/dimension.txt", "w") as f:
    f.write("DesignQA Results\n")
    f.write("Subset: Dimension\n")
    f.write(f"Num Questions: {len(J)}\n")
    f.write(f"ACC: {acc_macro:.6f}\n")
print("Score file → /content/dimension.txt")